In [31]:
from datasets import load_dataset
import spacy

from pathlib import Path
from os import listdir
from os.path import isfile, join
import os

In [17]:
# download stories from here
# https://cs.nyu.edu/~kcho/DMQA/

# 114 out of over 300,000 stories are missing text

# makes datafiles from this code
# https://github.com/becxer/cnn-dailymail/blob/master/make_datafiles.py

In [32]:
# """Load the dataset using huggingface data_loader"""
# type this command in the command line for the venv for this cell to work
# jupyter nbextension enable --py widgetsnbextension --sys-prefix 
dataset = load_dataset('cnn_dailymail', '3.0.0')

OSError: Not enough disk space. Needed: 1.82 GiB (download: 558.32 MiB, generated: 1.28 GiB, post-processed: Unknown size)

In [27]:
# CNN_STORIES_PATH = Path("D:/__School/__Masters/____2021fALL/5290 nlp/stories/cnn/stories")
# DM_STORIES_PATH = Path("D:/__School/__Masters/____2021fALL/5290 nlp/stories/dailymail/stories")

In [26]:
# def grab_files(directory):
#     for name in os.listdir(directory):
#        full_path = os.path.join(directory, name)
#        if os.path.isdir(full_path):
#            for entry in grab_files(full_path):
#                yield entry
#        elif os.path.isfile(full_path):
#            yield full_path
#        else:
#            print('Unidentified name %s. It could be a symbolic link' % full_path)

In [28]:
#i = 0
#for file in grab_files(CNN_STORIES_PATH):
#    i+=1
#    print(file)
#    if i == 5:
#        break

In [29]:
#f = open(CNN_STORIES_PATH.joinpath("0001d1afc246a7964130f43ae940af6bc6c57f01.story"), "r")
#print(f.read())

In [30]:
#f = open(CNN_STORIES_PATH.joinpath("0002095e55fcbd3a2f366d9bf92a95433dc305ef.story"), "r")
#print(f.read())